# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
G. Martos  ->  G. Martos  |  ['G. Martos']
J. Li  ->  J. Li  |  ['J. Li']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 64 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.16959


extracting tarball to tmp_2504.16959... done.
Retrieving document from  https://arxiv.org/e-print/2504.17478
extracting tarball to tmp_2504.17478... done.
Retrieving document from  https://arxiv.org/e-print/2504.17482


extracting tarball to tmp_2504.17482... done.
Retrieving document from  https://arxiv.org/e-print/2504.17516


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2504.17482/sample631.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'letter_maintable_activity_example' from 'tmp_2504.17482/letter_maintable_activity_example.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'letter_maintable_params_example' from 'tmp_2504.17482/letter_maintable_params_example.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from

extracting tarball to tmp_2504.17516...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.17741


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2504.17516/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'radio_table' from 'tmp_2504.17516/radio_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2504.17741...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.16959-b31b1b.svg)](https://arxiv.org/abs/2504.16959) | **On Validating Angular Power Spectral Models for the Stochastic Gravitational-Wave Background Without Distributional Assumptions**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-04-25*|
|*Comments*| **|
|**Abstract**|            It is demonstrated that estimators of the angular power spectrum commonly used for the stochastic gravitational-wave background (SGWB) lack a closed-form analytical expression for the likelihood function and, typically, cannot be accurately approximated by a Gaussian likelihood. Nevertheless, a robust statistical analysis can be performed by extending the framework outlined in \cite{PRL} to enable the estimation and testing of angular power spectral models for the SGWB without specifying distributional assumptions. Here, the technical aspects of the method are discussed in detail. Moreover, a new, consistent estimator for the covariance of the angular power spectrum is derived. The proposed approach is applied to data from the third observing run (O3) of Advanced LIGO and Advanced Virgo.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17478-b31b1b.svg)](https://arxiv.org/abs/2504.17478) | **Prospective sensitivity of CTA on detection of evaporating primordial black holes**  |
|| C. Yang, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-25*|
|*Comments*| *6 pages, 3 figures*|
|**Abstract**|            As the lifetime of a black hole decreases, the energy of the Hawking radiation it emits increases, ultimately culminating in its disappearance through a powerful burst of gamma rays. For primordial black holes (PBHs) with an initial mass of $\sim 5\times10^{14}$ g, their lifespans are expected to end in the present epoch. Detecting such PBH bursts would provide compelling evidence of their existence. The Cherenkov Telescope Array (CTA) has the potential to observe these bursts at the high-energy end of the gamma-ray spectrum. To investigate this possibility, we conduct a study to evaluate the sensitivity of CTA to the local burst rate density of PBHs. Our results suggest that during a 5-year observational campaign, CTA could exclude a local burst rate density exceeding $\sim 36\ \mathrm{pc}^{-3}\ \mathrm{yr}^{-1}$, which represents an improvement of one order of magnitude over the upper limit set by the Large High Altitude Air Shower Observatory (LHAASO). In addition, we propose an observation strategy optimized for detecting PBH bursts.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17482-b31b1b.svg)](https://arxiv.org/abs/2504.17482) | **A New Age-Activity Relation For Solar Analogs that Accounts for Metallicity**  |
|| G. Carvalho-Silva, et al. -- incl., <mark>G. Martos</mark> |
|*Appeared on*| *2025-04-25*|
|*Comments*| *10 pages, 4 figures*|
|**Abstract**|            Determining stellar ages is challenging, particularly for cooler main-sequence stars. Magnetic evolution offers an observational alternative for age estimation via the age-chromospheric activity (AC) relation. We evaluate the impact of metallicity on this relation using near one-solar-mass stars across a wide metallicity range. We analyze a sample of 358 solar-type stars with precise spectroscopic parameters determined through a line-by-line differential technique and with ages derived using Yonsei-Yale isochrones. We measured chromospheric activity (S-index) using high-quality HARPS spectra, calibrated to the Mount Wilson system, and converted to the $R^{\prime}_{\mathrm HK}(T_{\mathrm{eff}})$ index with a temperature-based photospheric correction. Our findings show that the AC relation for $R^{\prime}_{\mathrm HK}(T_{\mathrm{eff}})$ is strongly influenced by metallicity. We propose a new age-activity-metallicity relation for solar-type main-sequence (MS) stars ($\log{g} \gtrsim 4.2 $) with temperatures 5370 $\lesssim$ $T_{\mathrm{eff}}$ $\lesssim$ 6530 K and metallicities from -0.7 to +0.3 dex. We show that taking metallicity into account significantly enhances chromospheric ages' reliability, reducing the residuals' root mean square (RMS) relative to isochronal ages from 2.6 Gyr to 0.92 Gyr. This reflects a considerable improvement in the errors of chromospheric ages, from 53\% to 15\%. The precision level achieved in this work is also consistent with previous age-activity calibration from our group using solar twins.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17516-b31b1b.svg)](https://arxiv.org/abs/2504.17516) | **EP250108a/SN 2025kg: A Broad-Line Type Ic Supernova Associated with a Fast X-ray Transient Showing Evidence of Extended CSM Interaction**  |
|| G. P. Srinivasaragavan, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-25*|
|*Comments*| *21 pages, 11 Figures, Submitted to ApJ Letters, Comments Welcome*|
|**Abstract**|            We present optical, radio, and X-ray observations of EP250108a/SN 2025kg, a broad-line Type Ic supernova (SN Ic-BL) accompanying an Einstein Probe (EP) fast X-ray transient (FXT) at $z=0.176$. EP250108a/SN 2025kg possesses a double-peaked optical light curve and its spectrum transitions from a blue underlying continuum to a typical SN Ic-BL spectrum over time. We fit a radioactive decay model to the second peak of the optical light curve and find SN parameters that are consistent with the SNe Ic-BL population, while its X-ray and radio properties are consistent with those of low-luminosity GRB (LLGRB) 060218/SN 2006aj. We explore three scenarios to understand the system's multi-wavelength emission -- (a) SN ejecta interacting with an extended circumstellar medium (CSM), (b) the shocked cocoon of a collapsar-driven jet choked in its stellar envelope, and (c) the shocked cocoon of a collapsar-driven jet choked in an extended CSM. All three models can explain the optical light curve and are also consistent with the radio and X-ray observations. We favor models (a) and (c) because they self-consistently explain both the X-ray prompt emission and first optical peak, but we do not rule out model (b). From the properties of the first peak in models (a) and (c), we find evidence that EP250108a/SN 2025kg interacts with an extended CSM, and infer an envelope mass $M_{\rm e} \sim 0.1\,\rm M_\odot$ and radius $R_{\rm e} \sim 4 \times 10^{13}$ cm. EP250108a/SN 2025kg's multi-wavelength properties make it a close analog to LLGRB 060218/SN 2006aj, and highlight the power of early follow-up observations in mapping the environments of massive stars prior to core collapse.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17741-b31b1b.svg)](https://arxiv.org/abs/2504.17741) | **Multi-messenger standard-siren cosmology for third-generation gravitational-wave detectors: Considering observations of gamma-ray bursts and kilonovae**  |
|| T. Han, J.-F. Zhang, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-25*|
|*Comments*| *21 pages, 14 figures*|
|**Abstract**|            In the third-generation (3G) gravitational-wave (GW) detector era, GW multi-messenger observations for binary neutron star merger events can exert great impacts on exploring the cosmic expansion history. Extending the previous work, we explore the potential of 3G GW standard siren observations in cosmological parameter estimation by considering their associated electromagnetic (EM) counterparts, including $\gamma$-ray burst (GRB) coincidence observations by the Gravitational wave high-energy Electromagnetic Counterpart All-sky Monitor and GW-triggered target-of-opportunity observations of kilonovae by different optical survey projects. During an assumed 10-year observation, we predict that the number of detectable GW-kilonova events is $\sim 4900$ with redshifts below $\sim 0.4$ under GW network and Large Synoptic Survey Telescope in the $i$ band, which is three times more than that of GW-GRB detections. For the cosmological analysis, we find that with the inclusion of GW-kilonova detections, the constraints on cosmological parameters from GW-EM detections are significantly improved compared to those from GW-GRB detections. In particular, GW-EM detections can tightly constrain the Hubble constant with a precision ranging from $0.076\%$ to $0.034\%$. Moreover, GW multi-messenger observations could effectively break the cosmological parameter degeneracies generated by the mainstream EM observations, CMB+BAO+SN (CBS). The combination of CBS and GW-EM can tightly constrain the equation of state parameters of dark energy $w$ in the $w$CDM model and $w_0$ in the $w_0w_a$CDM model with precisions of $0.72\%$ and $0.99\%$, respectively, meeting the standard of precision cosmology. In conclusion, GW multi-messenger observations could play a crucial role in helping solve the Hubble tension and probing the fundamental nature of dark energy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

450  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
